In [15]:
from openai import OpenAI
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import os
import warnings

warnings.filterwarnings('ignore')

In [16]:
# 1. Set your OpenAI API key
client = OpenAI(api_key="")

In [17]:
# 2. Define a function to send prompt to OpenAI
def analyze_text_prompt_openai(query):
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that outputs only clean C++ code without any explanations."},
            {"role": "user", "content": query}
        ]
    )
    generated_text = response.choices[0].message.content
    return query, generated_text

In [18]:
queries = [
    """
You are tasked to generate a complete, compilable C++ file for a CPU cache eviction policy for ChampSim CRC2.

DO NOT output any English text, description, or explanation.
ONLY output C++ code.

You are free to design **any valid eviction policy** — it could be based on:
- LRU (Least Recently Used)
- Random
- LFU (Least Frequently Used)
- LFU+LRU hybrid
- Custom heuristics
- Anything that is internally consistent

However, you must strictly satisfy the following rules:

- Correctly match the ChampSim CRC2 interface:
  - `void InitReplacementState()`
  - `uint32_t GetVictimInSet(uint32_t cpu, uint32_t set, const BLOCK *current_set, uint64_t PC, uint64_t paddr, uint32_t type)`
  - `void UpdateReplacementState(uint32_t cpu, uint32_t set, uint32_t way, uint64_t paddr, uint64_t PC, uint64_t victim_addr, uint32_t type, uint8_t hit)`
  - `void PrintStats_Heartbeat()`
  - `void PrintStats()`
- Maintain any necessary per-line metadata (like access timestamps, access counters, etc.) appropriately
- Assume flat cache structure:
  - Number of sets = `NUM_SET`
  - Associativity = `ASSOC`
- At the top of your code, manually define:
  - `#define NUM_SET 2048`
  - `#define ASSOC 16`
- Only use standard C++11 or newer features
- Output pure C++ between `%%` markers only (no markdown, no ```cpp)

Below is an **example starter structure** — you must complete it based on the policy you choose:

%%
#include "champsim_crc2.h"
#include <vector>
#include <cstdint>
#include <climits>
#include <cstdlib>

#define NUM_SET 2048
#define ASSOC 16

// (Your metadata structs here)

void InitReplacementState() {
    // (Your initialization code here)
}

uint32_t GetVictimInSet(uint32_t cpu, uint32_t set, const BLOCK* current_set, uint64_t PC, uint64_t paddr, uint32_t type) {
    // (Your victim selection logic here)
}

void UpdateReplacementState(uint32_t cpu, uint32_t set, uint32_t way, uint64_t paddr, uint64_t PC, uint64_t victim_addr, uint32_t type, uint8_t hit) {
    // (Your state update logic here)
}

void PrintStats_Heartbeat() {
    // (Heartbeat stats here)
}

void PrintStats() {
    // (Final stats here)
}
%%
"""
]

In [19]:
# 4. Output directory setup
OUTPUT_DIR = ""
CS_DIR = ""
os.makedirs(OUTPUT_DIR, exist_ok=True)

results = []

In [20]:
# 5. Run the queries through OpenAI and save outputs
for idx, query in enumerate(tqdm(queries, desc="Evaluating Queries"), start=1):
    prompt, response = analyze_text_prompt_openai(query)

    # Clean if prompt is echoed
    if response.startswith(prompt):
        response = response[len(prompt):]

    # Extract C++ code between %%
    code_start = response.find("%%") + 2
    code_end = response.rfind("%%")
    if code_start == -1 or code_end == -1:
        extracted_code = response.strip()
    else:
        extracted_code = response[code_start:code_end].strip()

    # Save the C++ code file
    cpp_file_path = os.path.join(OUTPUT_DIR, f"replacement_policy_{idx}.cc")
    with open(cpp_file_path, "w") as f:
        f.write(extracted_code)

    cpp_file_path = os.path.join(CS_DIR, f"replacement_policy_{idx}.cc")
    with open(cpp_file_path, "w") as f:
        f.write(extracted_code)

    # Log the full response too
    results.append((query, response))

Evaluating Queries: 100%|██████████| 1/1 [00:19<00:00, 19.72s/it]


In [21]:
# 6. Save a CSV file logging all responses
df = pd.DataFrame(results, columns=["Query", "FullResponse"])
csv_path = os.path.join(OUTPUT_DIR, "OpenAI_RAG_Evals.csv")
df.to_csv(csv_path, index=False)

print(f"Testing complete. C++ files saved in {OUTPUT_DIR}")

✅ Testing complete. C++ files saved in /content/drive/MyDrive/GenAI Project/outputs


In [22]:
import subprocess

# Step 1: Set your existing .cc filename (without .cc extension)
cpp_filename = f"replacement_policy_1"
print(f"Using file: {cpp_filename}.cc")

# Step 2: Move to your project directory
%cd ""

# Step 3: Compile your existing .cc file with -Iinc and handle errors
compile_command = f"g++ -Wall --std=c++11 -Iinc -o {cpp_filename}-config1 example/{cpp_filename}.cc lib/config1.a"
compile_result = subprocess.run(compile_command, shell=True, capture_output=True, text=True)

if compile_result.returncode != 0:
    print("Compilation failed!")
    print("Error log:")
    print(compile_result.stderr)
else:
    print(f"{cpp_filename}-config1 compiled successfully.")

    # Step 4: Run the simulation and save output to a text file
    run_command = f"./{cpp_filename}-config1 -warmup_instructions 1000000 -simulation_instructions 10000000 -traces trace/astar_313B.trace.gz > ../outputs/{cpp_filename}_run_output.txt"
    run_result = subprocess.run(run_command, shell=True)

    if run_result.returncode != 0:
        print("Simulation failed!")
    else:
        print(f"Simulation finished. Output saved to outputs/{cpp_filename}_run_output.txt")


✅ Using file: replacement_policy_1.cc
/content/drive/MyDrive/GenAI Project/ChampSim_CRC2
✅ replacement_policy_1-config1 compiled successfully.
✅ Simulation finished. Output saved to outputs/replacement_policy_1_run_output.txt
